In [1]:
import datetime as dt
from utility import (
    read_sql,
    fd_basicinfo,
    fd_typeclass,
    nearest_report_date
)
import pandas as pd

In [2]:
# 获取回溯的报告日

today = dt.datetime.today().strftime("%Y%m%d")

In [3]:
# 获取所有目标股票型基金（清算结束日期晚于当前日）

security_ids = read_sql(f"""
select SECURITYID from TQ_FD_TYPECLASS 
WHERE
    ISVALID = 1 AND
    L1CODE = 1 AND
    (ENDDATE>='{today}' or ENDDATE = '19000101') 
ORDER BY SECURITYID;
""")["SECURITYID"].unique().tolist()

In [4]:
basic_info = fd_basicinfo(security_ids)
type_info = fd_typeclass(security_ids)

In [5]:
df = pd.merge(type_info, basic_info, on="SECURITYID")

In [10]:
df.L3NAME.unique()

array(['标准指数股票型基金', '增强指数股票型基金', 'ETF联接基金', '其他行业股票型基金',
       '偏股型基金（股票上下限60%-95%）', '标准股票型基金', '医药行业股票型基金', '偏债型基金',
       '股票型分级子基金(优先份额)', '股票型分级子基金(进取份额)', '封闭式标准股票型基金'], dtype=object)

In [19]:
# 分类
df.loc[(df["L3NAME"].isin(["其他行业股票型基金", "偏股型基金（股票上下限60%-95%）", '标准股票型基金', '医药行业股票型基金', '封闭式标准股票型基金'])), "股票型（子类）"] = "普通股票型基金"
df.loc[(df["L3NAME"].isin(["标准指数股票型基金"])), "股票型（子类）"] = "被动指数型基金"
df.loc[(df["L3NAME"].isin(["增强指数股票型基金"])), "股票型（子类）"] = "增强指数型基金"
# df.loc[(df["FDNATURE"] != "ETF"), "货币型（子类）"] = "场外货币型基金"

In [20]:
df.groupby("股票型（子类）").count()

,SECURITYID,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE
股票型（子类）,,,,,,,,,,,,
增强指数型基金,63,63,63,63,63,63,63,63,45,63,63,63
普通股票型基金,217,217,217,217,217,217,217,217,66,217,217,217
被动指数型基金,344,344,344,344,344,344,344,344,215,344,344,344


In [16]:
df

,SECURITYID,L1CODE,L1NAME,L2CODE,L2NAME,L3CODE,L3NAME,FDNAME,SNAMECOMP,FSYMBOL,FDNATURE,INVESTSTYLE,股票型（子类）
0,1030000030,1,股票基金,1.2,指数股票型基金,1.2.1,标准指数股票型基金,国泰沪深300指数证券投资基金,国泰沪深300指数,020011,证券投资基金,指数型,被动指数型基金
1,1030000034,1,股票基金,1.2,指数股票型基金,1.2.2,增强指数股票型基金,华安MSCI中国A股指数增强型证券投资基金,None,040002,证券投资基金,指数型,NaN
2,1030000045,1,股票基金,1.2,指数股票型基金,1.2.1,标准指数股票型基金,博时裕富沪深300指数证券投资基金,博时沪深300指数,050002,证券投资基金,指数型,被动指数型基金
3,1030000045,1,股票基金,1.2,指数股票型基金,1.2.1,标准指数股票型基金,博时裕富沪深300指数证券投资基金,博时沪深300指数,050002,证券投资基金,指数型,被动指数型基金
4,1030000096,1,股票基金,1.2,指数股票型基金,1.2.2,增强指数股票型基金,易方达上证50指数增强型证券投资基金,易方达上证50增强,110003,证券投资基金,指数型,NaN
...,...,...,...,...,...,...,...,...,...,...,...,...,...
933,1030005090,1,股票基金,1.2,指数股票型基金,1.2.1,标准指数股票型基金,富国中证煤炭指数分级证券投资基金,富国中证煤炭指数分级,161032,证券投资基金,指数型,被动指数型基金
934,1030005095,1,股票基金,1.2,指数股票型基金,1.2.1,标准指数股票型基金,信诚中证信息安全指数分级证券投资基金,信诚中证信息安全指数分级,165523,证券投资基金,指数型,被动指数型基金
935,1030005100,1,股票基金,1.2,指数股票型基金,1.2.1,标准指数股票型基金,信诚中证智能家居指数分级证券投资基金,信诚中证智能家居指数分级,165524,证券投资基金,指数型,被动指数型基金
936,1030005248,1,股票基金,1.2,指数股票型基金,1.2.1,标准指数股票型基金,华安创业板50指数分级证券投资基金,华安创业板50指数分级,160420,证券投资基金,指数型,被动指数型基金
